In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install -U ipywidgets

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
# yolo predict model=yolo11n.pt source='https://ultralytics.com/images/bus.jpg'

In [ ]:
#Remove all files

import os
import shutil

# Specify the folder path
folder_path = '/kaggle/working/'

# Remove all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # Remove file or link
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)  # Remove directory and its contents
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

print("All files have been removed from the folder.")


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x-seg.pt")

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="xTo8OzKt3gqUQENvd7gp")
project = rf.workspace("damage-sverity").project("flowers-krbsa")
version = project.version(2)
dataset = version.download("yolov11")

# Training on GPU (Option 01)

In [ ]:
# Train the model
train_results = model.train(
    data="/kaggle/working/Flowers-2/data.yaml",  # path to dataset YAML
    epochs=600,  # number of training epochs
    imgsz=640,  # training image size
    device=0,
    batch=4,
    workers=4,
    seed=101,
    
    # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Training on GPU (Option 02)

In [ ]:
!yolo segment train model=yolo11x-seg.pt data='/data/data_modified.yaml' workers=8 batch=8 epochs=300 seed=101 device=0 save_period=40 #patience=600 #name=yolov8_car_damage_296_Classes

# Continue Training

In [ ]:
!yolo segment train model=weights/best.pt data='/data/data_modified.yaml' resume=True workers=8 batch=8 epochs=600 seed=101 device=0 save_period=40 #patience=600 #name=yolov8_car_damage_296_Classes

# Saving the model

In [ ]:
# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

# Inference (Option 01)

In [ ]:
# Evaluate model performance on the validation set
metrics = model.val()

# Perform object detection on an image
results = model("/kaggle/working/Flowers-2/valid/images/IMG_3166_png_jpg.rf.192d29b990828bf0a453106caea9490a.jpg")
results[0].show()

# Inference (Option 02)

In [ ]:
!yolo predict model=yolo11x-seg.pt source='/images/IMG_3190a.jpg'

# Check labels from the model

In [ ]:
model.names

# Change Labels

In [ ]:
import torch

model = torch.load(r"c:\Users\Sakib Ahmed\Desktop\model_car_parts.pt", map_location="cpu")

model["model"].names

In [ ]:
# Dictionary mapping old class names to new class names
car_parts_class_mapping = {
    0: 'Alloy Rim Back Left Side',
    1: 'Alloy Rim Back Right Side',
    2: 'Alloy Rim Front Left Side',
    3: 'Alloy Rim Front Right Side',
    4: 'Antenna',
    5: 'Back Bumper',
    6: 'Back Logo',
    7: 'Back Wheel Left Side',
    8: 'Back Wheel Right Side',
    9: 'Back Window Left Side',
    10: 'Back Window Right Side',
    11: 'Back Windshield',
    12: 'Backdoor Left Side',
    13: 'Backdoor Right Side',
    14: 'Brake Light',
    15: 'Exterior Door Handle Back Left Side',
    16: 'Exterior Door Handle Back Right Side',
    17: 'Exterior Door Handle Front Left Side',
    18: 'Exterior Door Handle Front Right Side',
    19: 'Fender Left Side',
    20: 'Fender Right Side',
    21: 'Fog Light Back Left Side',
    22: 'Fog Light Back Right Side',
    23: 'Fog Light Front Left Side',
    24: 'Fog Light Front Right Side',
    25: 'Front Arch Panel Left Side',
    26: 'Front Arch Panel Right Side',
    27: 'Front Logo',
    28: 'Front Wheel Left Side',
    29: 'Front Wheel Right Side',
    30: 'Front Window Left Side',
    31: 'Front Window Right Side',
    32: 'Front Bumper',
    33: 'Front Side Bumper Left Side',
    34: 'Front Side Bumper Right Side',
    35: 'Front Door Left Side',
    36: 'Front Door Right Side',
    37: 'Fuel Cap',
    38: 'Grille',
    39: 'Head Light Left Side',
    40: 'Head Light Right Side',
    41: 'Hood',
    42: 'Indicator Light Left Side',
    43: 'Indicator Light Right Side',
    44: 'License Plate',
    45: 'Molding Left Side',
    46: 'Molding Right Side',
    47: 'Quarter Panel Left Side',
    48: 'Quarter Panel Right Side',
    49: 'Quarter Glass Left Side',
    50: 'Quarter Glass Right Side',
    51: 'Rear Arch Panel Left Side',
    52: 'Rear Arch Panel Right Side',
    53: 'Rear Wiper',
    54: 'Rear Side Bumper Left Side',
    55: 'Rear Side Bumper Right Side',
    56: 'Rocker Panel',
    57: 'Roof',
    58: 'Roof Rail',
    59: 'Side Rocker Panel Right Side',
    60: 'Side Rocker Panel Left Side',
    61: 'Side Mirror Left Side',
    62: 'Side Mirror Right Side',
    63: 'Tail Light Left Side',
    64: 'Tail Light Right Side',
    65: 'Trunk',
    66: 'Wind Shield',
    67: 'Windshield Wiper',
    68: 'Spoiler'
}

model["model"].names = car_parts_class_mapping

In [ ]:
model["model"].names

# Save the revised model

In [ ]:
torch.save(model, "model_car_parts.pt")